# Neural Network 정의
nn_id : Neural Network ID<br>
biz_cate : Business 대분류<br>
biz_sub_cate : Business 소분류<br>
nn_title : Neural Network Title<br>
nn_desc : Neural Network Description<br>
use_flag : 사용여부<br> 
dir : 문제유형<br>
config : Custom 여부 

In [12]:
import requests
import json, os

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")

nn_id = "nn00002"
nn_wf_ver_id ="1"

# insert submenu info
resp = requests.post('http://' + url + '/api/v1/type/wf/target/menu/netconf/submenu/',
                     json={
                        "wf_task_submenu_id": "word_to_vec",
                        "wf_task_submenu_name": "word_to_vec",
                        "wf_task_submenu_desc": "word_to_vec",
                        "wf_node_class_path": "cluster.neuralnet.neuralnet_node_w2v",
                        "wf_node_class_name": "NeuralNetNodeWord2Vec"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : word_to_vec
evaluation result : nn00002


# Work Flow Version 정의
nn_def_list_info_nn_id :<br>
nn_wf_ver_info : Work Flow Version Description<br>
condition :<br>
active_flag : 사용여부
master_nn_ver_wflist_info insert

In [13]:
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                     json={
                         "nn_def_list_info_nn_id": "",
                         "nn_wf_ver_info": "test version info",
                         "condition": "1",
                         "active_flag": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : nn00002


In [14]:

# update workflow info
resp = requests.put('http://' + url + '/api/v1/type/common/target/nninfo/' + nn_id + '/version/',
                     json={
                         "nn_wf_ver_id": nn_wf_ver_id,
                         "nn_def_list_info_nn_id": "",
                         "nn_wf_ver_info": "test version info",
                         "condition": "1",
                         "active_flag": "Y"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

# Work Flow Node  Initialize
type : Neural Network Type<br>
1. Data Node<br>
2. Neural Network Node<br>
3. Evaluation Node<br>

In [ ]:
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/ '+ nn_id + '/wfver/1/',
                     json={
                         "type": "word2vec"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# Data Node Config
type : Image Source Type<br>
labels : Classfication Labels<br>
source_path : Image Source Path<br>
preprocess : Image Preprocess Config<br>
store_path : Image Store Path

In [ ]:
def get_all_files(directory) :

    if not os.path.exists(directory):
        os.makedirs(directory)

    return_dict = {}
    count = 0
    for root, directories, files in os.walk(directory):
        for filename in files:
            filepath = os.path.join(root, filename)
            return_dict[''.join(['file', str(count)])] = open(filepath, 'rb')
            count = count + 1
    return return_dict
train_files =  get_all_files('/home/dev/text/')


In [ ]:
# update source_info
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/' + nn_id + '/ver/1/node/data_node/',
                     json={
                         "source_server": "local",
                         "source_sql": "all",
                         "max_sentence_len" : 50
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))



# Select Preprocess Type
preprocess : kkma, twiter, mecab, nltk<br>

In [18]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/pre/nnid/' + nn_id + '/ver/1/node/data_node/',
                     json={
                         "preprocess":  "mecab",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : mecab


In [19]:
# update store_path
resp = requests.put('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/store/nnid/' + nn_id + '/ver/1/node/data_node/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/nn00002/1/data_node


In [20]:
# check properties are updated well
resp = requests.get('http://' + url + '/api/v1/type/wf/state/textdata/src/local/form/raw/prg/source/nnid/'+ nn_id +'/ver/1/node/data_node/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'type': 'csv', 'multi_node_flag': True, 'preprocess': 'mecab', 'max_sentence_len': 10, 'source_server': 'local', 'store_path': '/hoya_str_root/nn00002/1/data_node', 'source_type': 'local', 'source_sql': 'all', 'source_parse_type': 'raw', 'source_path': '/hoya_src_root/nn00002/1/data_node'}


# Neural Network Node Config
model_path : Model Save Path<br>
window_size : 문장 내의 현재 단어와 예측 단어 사이의 최대 거리.<br>
vector_size : 백터사이즈.<br>
batch_size : Batch Size (Vocabary Distance)<br>
iter : Trainning Iteration<br>

In [21]:

# update source_info
# set netconf_node in NN_WF_NODE_INFO
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/w2v/nnid/' + nn_id + '/ver/' + nn_wf_ver_id + '/node/netconf_node/',
                     json={
                        "model_path" : "test",
                        "window_size" : 5,
                        "vector_size" : 100,
                        "batch_size" : 100,
                        "iter" : 5,
                        "min_count" : 1
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'vector_size': 100, 'batch_size': 100, 'window_size': 5, 'min_count': 1, 'iter': 5, 'model_path': '/hoya_model_root/nn00002/1/netconf_node'}


# Training

In [22]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/'+nn_wf_ver_id+'/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : {'result': "'store_path'", 'status': '404'}


# Prediction
type :  sim
val_1 : [철강,권오준,연구원,포스코]


포스코-적자
철강-포스코


In [ ]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/w2v/nnid/' + nn_id + '/ver/active/',
                     json={
                         "type": "sim",
                         "val_1":["철강"],
                         "val_2":[""]
                     }
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


In [ ]:
type :  similarity
val_1 : 철강
val_2 : 기술

In [ ]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/w2v/nnid/' + nn_id + '/ver/active/',
                     json={
                        "type": "similarity",
                        "val_1": ["회장"], 
                        "val_2": ["적자"]
                     }
                     )
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))
